## Load the Dataset and Pick a Column (social media dataset)

In [6]:
import pandas as pd

df = pd.read_csv("SocialMedia.csv")
docs = df['Text'].dropna().astype(str).tolist()

# show first 10 row
for i, doc in enumerate(docs[:10]):
    print(f"{i+1}: {doc}")


1:  Enjoying a beautiful day at the park!              
2:  Traffic was terrible this morning.                 
3:  Just finished an amazing workout! 💪               
4:  Excited about the upcoming weekend getaway!        
5:  Trying out a new recipe for dinner tonight.        
6:  Feeling grateful for the little things in life.    
7:  Rainy days call for cozy blankets and hot cocoa.   
8:  The new movie release is a must-watch!             
9:  Political discussions heating up on the timeline.  
10:  Missing summer vibes and beach days.               


## Define Queries

In [33]:
queries = [
    "beautiful day",
    "Enjoying every moment",
    "the sky with",
    "Gratitude for the support received",
    "Compassion shown through acts of kindness in the community",
    "Determination burning like a wildfire, overcoming obstacles, turning dreams into reality",
    "Overwhelmed by the cacophony of expectations",
    "worst experience ever",
    "Eyes wide open in the night",
    "In the celebration",
    "Wandering through the historical streets of Kyoto, each step a journey into the heart of Japan's traditions"
]

## Biword Indexing

In [34]:
import re

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text.split()

def build_biword_index(docs):
    biword_index = {}
    for doc_id, doc in enumerate(docs):
        words = preprocess_text(doc)
        for i in range(len(words) - 1):
            biword = (words[i], words[i + 1])
            if biword in biword_index.keys():
                biword_index[biword] += [doc_id]
            else:
                biword_index[biword] = [doc_id]
    return biword_index

def search_biword_index(biword_index, query):
    words = preprocess_text(query)
    if len(words) < 2:
        return set()  
    
    query_biwords = [(words[i], words[i + 1]) for i in range(len(words) - 1)]

    result_docs = None
    for biword in query_biwords:
        if biword in biword_index:
            if result_docs is None:
                result_docs = set(biword_index[biword])  
            else:
                result_docs = result_docs.intersection(biword_index[biword])
        else:
            return set()

    return result_docs if result_docs else set()


# build
biword_index = build_biword_index(docs)
# max_biword = sorted(biword_index.items(), key=lambda item: len(item[1]), reverse=True)
# print(max_biword)

# Perform search
for query in queries :
    result = search_biword_index(biword_index, query)

    print("Query:", query)
    print("Number of Matching docs:", len(result))
    print("Matching Document IDs:", result)
    print("Results found:")
    for doc_id in result:
        print("Document:", doc_id, "->", docs[doc_id])
    print('\n')    
    

Query: beautiful day
Number of Matching docs: 2
Matching Document IDs: {0, 82}
Results found:
Document: 0 ->  Enjoying a beautiful day at the park!              
Document: 82 ->  Sending love to all my followers on this beautiful day! ❤️ 


Query: Enjoying every moment
Number of Matching docs: 1
Matching Document IDs: {84}
Results found:
Document: 84 ->  Enjoying every moment of this trip—pure enjoyment!      


Query: the sky with
Number of Matching docs: 5
Matching Document IDs: {268, 496, 505, 282, 255}
Results found:
Document: 268 ->  Floating on clouds of inspiration, an artist painting the sky with strokes of creativity, creating a masterpiece of dreams. 
Document: 496 -> Journeying through the serenity of Santorini, where each sunset paints the sky with hues of tranquility. 
Document: 505 -> At the summit of Mount Fuji, a breathtaking sunrise that paints the sky with hues of accomplishment. 
Document: 282 ->  Floating on clouds of inspiration, an artist painting the sky with str

## Positional Index

In [44]:
from collections import defaultdict

def preprocess_text(text):
    import re
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text.split()

def build_positional_index(documents):

    positional_index = defaultdict(list)

    for doc_id, document in enumerate(documents):
        words = preprocess_text(document)
        for pos, word in enumerate(words):
            # Append the position to the term's entry
            if positional_index[word] and positional_index[word][-1][0] == doc_id:
                positional_index[word][-1][1].append(pos)
            else:
                positional_index[word].append((doc_id, [pos]))

    return positional_index

def search_phrase(positional_index, query):

    words = preprocess_text(query)
    if not words:
        return set()

    # Find the list of (doc_id, positions) for each word in the query
    word_positions = [positional_index.get(word, []) for word in words]
    if not all(word_positions):
        return set()

    # Find documents containing all words with correct relative positions
    result_docs = set(doc_id for doc_id, _ in word_positions[0])
    for i in range(1, len(words)):
        next_result_docs = set()
        for doc_id, positions in word_positions[i]:
            for prev_doc_id, prev_positions in word_positions[i - 1]:
                if doc_id == prev_doc_id:
                    if any(pos + 1 in positions for pos in prev_positions):
                        next_result_docs.add(doc_id)
                        break
        result_docs &= next_result_docs
        if not result_docs:
            return set()

    return result_docs


# build
positional_index = build_positional_index(docs)

# Perform search
for query in queries:
    result = search_phrase(positional_index, query)

    print("Query:", query)
    print("Number of Matching docs:", len(result))
    print("Matching Document IDs:", result)
    print("Results found:")
    for doc_id in result:
        print("Document:", doc_id, "->", docs[doc_id])
    print('\n')

Query: beautiful day
Number of Matching docs: 2
Matching Document IDs: {0, 82}
Results found:
Document: 0 ->  Enjoying a beautiful day at the park!              
Document: 82 ->  Sending love to all my followers on this beautiful day! ❤️ 


Query: Enjoying every moment
Number of Matching docs: 1
Matching Document IDs: {84}
Results found:
Document: 84 ->  Enjoying every moment of this trip—pure enjoyment!      


Query: the sky with
Number of Matching docs: 5
Matching Document IDs: {268, 496, 505, 282, 255}
Results found:
Document: 268 ->  Floating on clouds of inspiration, an artist painting the sky with strokes of creativity, creating a masterpiece of dreams. 
Document: 496 -> Journeying through the serenity of Santorini, where each sunset paints the sky with hues of tranquility. 
Document: 505 -> At the summit of Mount Fuji, a breathtaking sunrise that paints the sky with hues of accomplishment. 
Document: 282 ->  Floating on clouds of inspiration, an artist painting the sky with str

## Skip Pointer

In [36]:
import math
from collections import defaultdict

def preprocess_text(text):
    import re
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text.split()

def add_skip_pointers(postings):
    n = len(postings)
    
    # The ideal distance between skip pointers is sqrt(n). This is based on optimal skip distance in IR theory
    skip_len = int(math.sqrt(n))
    skips = {}
    if skip_len < 1:
        return skips
    for i in range(0, n, skip_len):
        j = i + skip_len
        if j < n:
            skips[i] = j
    return skips

def intersect_with_skips(p1, p2):
    skips1 = add_skip_pointers(p1)
    skips2 = add_skip_pointers(p2)
    ans = []
    i = j = 0
    while i < len(p1) and j < len(p2):
        if p1[i] == p2[j]:
            ans.append(p1[i])
            i += 1
            j += 1
        elif p1[i] < p2[j]:
            if i in skips1 and p1[skips1[i]] <= p2[j]:
                i = skips1[i]
            else:
                i += 1
        else:
            if j in skips2 and p2[skips2[j]] <= p1[i]:
                j = skips2[j]
            else:
                j += 1
    return ans

def build_skip_pointer_index(positional_index):    
    skip_index = {}
    for term, plist in positional_index.items():
        docs = [doc_id for doc_id, _ in plist]
        docs.sort()
        skips = add_skip_pointers(docs)
        skip_index[term] = {
            'postings': docs,
            'skips': skips
        }
    return skip_index

def search_phrase_with_skips(positional_index, skip_index, query):
    words = preprocess_text(query)
    if not words:
        return []

    if any(word not in skip_index for word in words):
        return []

    # Step 1: Intersect document lists using skip pointers
    result_docs = skip_index[words[0]]['postings']
    for word in words[1:]:
        result_docs = intersect_with_skips(result_docs, skip_index[word]['postings'])
        if not result_docs:
            return []

    # Step 2: Confirm positional matches in result_docs
    matches = []
    for doc_id in result_docs:
        positions_lists = []
        for word in words:
            positions = next((pos for d, pos in positional_index[word] if d == doc_id), [])
            positions_lists.append(positions)

        for pos in positions_lists[0]:
            if all([(pos + i) in positions_lists[i] for i in range(1, len(words))]):
                matches.append(doc_id)
                break

    return matches

# build indexes 
positional_index = build_positional_index(docs)
skip_index = build_skip_pointer_index(positional_index)

# perform search
for query in queries:
    result = search_phrase_with_skips(positional_index, skip_index, query)

    print("Query:", query)
    print("Number of Matching docs:", len(result))
    print("Matching Document IDs:", result)
    print("Results found:")
    for doc_id in result:
        print("Document:", doc_id, "->", docs[doc_id])
    print('\n')


Query: beautiful day
Number of Matching docs: 2
Matching Document IDs: [0, 82]
Results found:
Document: 0 ->  Enjoying a beautiful day at the park!              
Document: 82 ->  Sending love to all my followers on this beautiful day! ❤️ 


Query: Enjoying every moment
Number of Matching docs: 1
Matching Document IDs: [84]
Results found:
Document: 84 ->  Enjoying every moment of this trip—pure enjoyment!      


Query: the sky with
Number of Matching docs: 5
Matching Document IDs: [255, 268, 282, 496, 505]
Results found:
Document: 255 ->  Soaring on the wings of a free spirit, unburdened by the chains of conformity, painting the sky with independence. 
Document: 268 ->  Floating on clouds of inspiration, an artist painting the sky with strokes of creativity, creating a masterpiece of dreams. 
Document: 282 ->  Floating on clouds of inspiration, an artist painting the sky with strokes of creativity, creating a masterpiece of dreams. 
Document: 496 -> Journeying through the serenity of S


## Summary Comparison Table:

| Feature               | Biword Retrieval           | Positional Index       | Skip Pointers                |
| --------------------- | -------------------------- | ---------------------- | ---------------------------- |
| Handles phrases?      | Yes (short phrases)        | Yes (any length)       | No (used to speed up search) |
| Supports long phrases | No                         |  Yes                   |  Not relevant directly       |
| Speed                 | Fast for 2-word phrases    |  Slower but flexible   |  Speeds up word search       |
| Storage               | Medium                     |  Higher                |  Helps large indexes         |
| Accuracy              | May have false positives   |  Exact                 |  Faster without loss         |





## Evaluation
#### True Positive and False Positive

In [ ]:
# Build all indexes and store results for evaluation
biword_index = build_biword_index(docs); biword_results = {}
positional_index = build_positional_index(docs); positional_results = {}
skip_index = build_skip_pointer_index(positional_index); skip_results = {}

for query in queries:
    p_result = search_phrase(positional_index, query)
    positional_results[query] = p_result
    b_result = search_biword_index(biword_index, query)
    biword_results[query] = b_result
    s_result = set(search_phrase_with_skips(positional_index, skip_index, query))
    skip_results[query] = s_result

print(f"{'Query':<61} | {'Pos TP':<8} {'Pos FP':<8} {'Biword TP':<10} {'Biword FP':<10} {'Skip TP':<10} {'Skip FP':<10}")
print('-'*120)

for i, query in enumerate(queries):
    ground = positional_results[query]
    p_tp = len(ground)
    p_fp = 0    # I checked and compute FP of positional index that is 0; Ground truth has no FP
    
    b = biword_results[query]
    b_tp = len(b & ground)
    b_fp = len(b - ground)
    
    s = skip_results[query]
    s_tp = len(s & ground)
    s_fp = len(s - ground)

    print(f"{i:<3}-{query[:55]:<58} | {p_tp:<8} {p_fp:<8} {b_tp:<10} {b_fp:<10} {s_tp:<10} {s_fp:<10}")



Query                                                         | Pos TP   Pos FP   Biword TP  Biword FP  Skip TP    Skip FP   
------------------------------------------------------------------------------------------------------------------------
0  -beautiful day                                              | 2        0        2          0          2          0         
1  -Enjoying every moment                                      | 1        0        1          0          1          0         
2  -the sky with                                               | 5        0        5          0          5          0         
3  -Gratitude for the support received                         | 1        0        1          0          1          0         
4  -Compassion shown through acts of kindness in the commun    | 1        0        1          0          1          0         
5  -Determination burning like a wildfire, overcoming obsta    | 1        0        1          0          1          0 